In [1]:
from datasets import load_dataset

training_dataset_raw = load_dataset("stanfordnlp/sst2", split="train")
validation_dataset_raw = load_dataset("stanfordnlp/sst2", split="validation")
test_dataset = load_dataset("stanfordnlp/sst2", split="test")

training_texts = training_dataset_raw["sentence"]
training_sentiment = training_dataset_raw["label"]
validation_texts = validation_dataset_raw["sentence"]
validation_sentiment = validation_dataset_raw["label"]

/Users/christianharjuno/anaconda3/envs/sentiment-analysis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

training_input = tokenizer(training_texts, padding=True,truncation=True,return_tensors="tf")
validation_input = tokenizer(validation_texts, padding=True,truncation=True,return_tensors="tf")

/Users/christianharjuno/anaconda3/envs/sentiment-analysis/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2025-04-28 18:08:22.636719: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-04-28 18:08:22.636849: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-28 18:08:22.636855: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-28 18:08:22.636877: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-28 18:08:22.636886: 

In [3]:
import tensorflow as tf
training_dataset = tf.data.Dataset.from_tensor_slices((
  dict(training_input),
  training_sentiment
  ))
validation_dataset = tf.data.Dataset.from_tensor_slices((
  dict(validation_input),
  validation_sentiment
))

training_dataset = training_dataset.shuffle(1000).batch(100)
validation_dataset = validation_dataset.batch(100)



In [4]:
import tensorflow as tf

if tf.config.list_physical_devices("GPU"):
  print("Using GPU")
else:
  print("Not using GPU")

Using GPU


In [5]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf


model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import tensorflow as tf

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)


model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])


In [7]:
import tensorflow as tf
tensorboard_callback = tf.keras.callbacks.TensorBoard("./logs", histogram_freq=1, write_graph=True, write_images=True, write_steps_per_second=True, update_freq="batch")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint("checkpoint.keras", monitor="val_loss", save_best_only=True, save_freq=1, save_weights_only=True)

In [8]:
history = model.fit(training_dataset, epochs=3, validation_data=validation_dataset, callbacks=[tensorboard_callback])

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2025-04-28 18:08:43.403752: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-04-28 18:08:44.149211: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


162/674 [======>.......................] - ETA: 23:36 - loss: 0.7510 - accuracy: 0.5534

KeyboardInterrupt: 

In [ ]:
model.save("progress.keras")